In [1]:
cd ../QC_DSP

/mnt/custom-file-systems/efs/fs-0efe0723c8fe23def_fsap-009815092600551f0/projects/QC_DSP/QC_DSP


In [2]:
import db, importlib
import random, string

In [18]:
importlib.reload(db)
customers = db.customers()
templates = db.templates()
campaigns = db.campaigns()

- Customer IDs, Campaign IDs and Template IDs must be unique wrt entity
- templates can be used in multiple campaigns
- components and compnent values need only be unique with respect to template

In [4]:
customers.update('test', 'test7', ['T_X1', 'T_X2'])
customers.update('test', 'test3', ['template_M1', 'T_J2', 'template_B3'])

{'ResponseMetadata': {'RequestId': 'GTMBHQAJ18CLTBBFQ2DTUERLCVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 04 Feb 2026 16:12:35 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'GTMBHQAJ18CLTBBFQ2DTUERLCVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [5]:
customers.get('test')

[{'customer_id': 'test',
  'template_ids': ['template_A1', 'template_2', 'template_3'],
  'timestamp': '2026-02-03 20:35:10.807370',
  'campaign_id': 'test1'},
 {'customer_id': 'test',
  'template_ids': ['template_B1', 'template_B2', 'template_B3'],
  'timestamp': '2026-02-03 20:24:59.249836',
  'campaign_id': 'test2'},
 {'customer_id': 'test',
  'template_ids': ['template_M1', 'T_J2', 'template_B3'],
  'timestamp': '2026-02-04 16:12:35.206687',
  'campaign_id': 'test3'},
 {'customer_id': 'test',
  'template_ids': ['T_X1', 'T_X2'],
  'timestamp': '2026-02-04 16:12:35.146752',
  'campaign_id': 'test7'}]

In [11]:
template_name = 'T_X1'
campaign_name = 'test7'

In [7]:
def random_string():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=4))


T = {'headline_text': {random_string(): ['Wool Socks', "Don't run in the house!"],
                       random_string(): ['Wool Socks', 'To keep you warm this winter!'],
                       random_string(): ['Warm Socks', 'Perfect Christmas Gift!']},
     'cta': {random_string(): 'Order Now!',
             random_string(): "Act Now!",
             random_string(): "Don't Delay!",
             random_string(): "While supplies last!"},
     'price': {random_string(): '50% Off',
               random_string(): '2 for 1',
               random_string(): '$6.99'},
     'theme': {random_string(): '#fff',
               random_string(): 'slateblue',
               random_string(): 'huntergreen'}}

for k in T.keys():
    templates.update(template_name, k, T[k])

In [195]:
T['Button'] = {random_string(): 'Round',
               random_string(): 'Square'}
for k in T.keys():
    templates.update('template_M1', k, T[k])

In [8]:
templates.get(template_name, 'price')

[{'template_id': 'T_X1',
  'possible_values': {'kd13': '50% Off', 'Zxc0': '2 for 1', 'Qb6H': '$6.99'},
  'timestamp': '2026-02-04 16:16:54.167330',
  'component_id': 'price'}]

In [12]:

def random_ad(template_id):
    template = templates.get(template_id)
    return '-'.join([random.choice([x for x in k['possible_values'].keys()]) for k in template])

In [13]:
# pick an ad_id to activate
campaigns.update(campaign_name, template_name, True, {'news_general': [random_ad(template_name) for z in range(4)], 
                                                      'fashion_trendy': [random_ad(template_name) for z in range(4)]})

{'ResponseMetadata': {'RequestId': '21UVAQC50K2C8NEV1SBDMBKBEBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 04 Feb 2026 16:46:33 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '21UVAQC50K2C8NEV1SBDMBKBEBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [14]:
active = campaigns.get(campaign_name)
active

[{'active': True,
  'segments': {'news_general': ['WdKj-Ja8S-Zxc0-hmNV',
    'WdKj-0WJq-Zxc0-hmNV',
    'BeZt-0WJq-Qb6H-RYn1',
    'BeZt-Ja8S-Qb6H-Z20W'],
   'fashion_trendy': ['WdKj-W1Yg-Qb6H-Z20W',
    'd8i1-Ja8S-Zxc0-hmNV',
    '9dpq-W1Yg-Zxc0-Z20W',
    'BeZt-0WJq-Zxc0-Z20W']},
  'campaign_id': 'test7',
  'timestamp': '2026-02-04 16:46:33.923309',
  'template_id': 'T_X1'}]

In [158]:
# can set all templates in a campaign or just one template in a campaign
campaigns.set(False, 'test3')

In [17]:
for a in active:
    template = templates.get(a['template_id'])
    for s in a['segments']:
        for ad_ids in a['segments'][s]:
            for ad_id, t in zip(ad_ids.split('-'), template):
                if a['active']:
                    print(a['template_id'], s, ad_id, t['component_id'], t['possible_values'][ad_id])


T_X1 news_general WdKj cta Don't Delay!
T_X1 news_general Ja8S headline_text ['Warm Socks', 'Perfect Christmas Gift!']
T_X1 news_general Zxc0 price 2 for 1
T_X1 news_general hmNV theme slateblue
T_X1 news_general WdKj cta Don't Delay!
T_X1 news_general 0WJq headline_text ['Wool Socks', "Don't run in the house!"]
T_X1 news_general Zxc0 price 2 for 1
T_X1 news_general hmNV theme slateblue
T_X1 news_general BeZt cta Order Now!
T_X1 news_general 0WJq headline_text ['Wool Socks', "Don't run in the house!"]
T_X1 news_general Qb6H price $6.99
T_X1 news_general RYn1 theme #fff
T_X1 news_general BeZt cta Order Now!
T_X1 news_general Ja8S headline_text ['Warm Socks', 'Perfect Christmas Gift!']
T_X1 news_general Qb6H price $6.99
T_X1 news_general Z20W theme huntergreen
T_X1 fashion_trendy WdKj cta Don't Delay!
T_X1 fashion_trendy W1Yg headline_text ['Wool Socks', 'To keep you warm this winter!']
T_X1 fashion_trendy Qb6H price $6.99
T_X1 fashion_trendy Z20W theme huntergreen
T_X1 fashion_trendy d8

In [20]:
db.clicks().get('T_X1')

[{'template_id': 'T_X1',
  'ad_id': 'WdKj-0WJq-Zxc0-hmNV',
  'campaign_id': 'test7',
  'segment': 'news_general',
  'timestamp': '2026-02-04 18:28:42.120422'},
 {'template_id': 'T_X1',
  'ad_id': 'WdKj-0WJq-Zxc0-hmNV',
  'campaign_id': 'test7',
  'segment': 'news_general',
  'timestamp': '2026-02-04 18:30:37.670271'}]